Database management with Django ORM is a powerful feature that allows you to interact with databases using Python code without writing raw SQL. 

Here's a detailed guide to managing databases in Django using its ORM:

## Defining Models

Models in Django define the structure of your database tables. Each model corresponds to a table in the database.

In [ ]:
# models.py

from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField(unique=True)

class Book(models.Model):
    title = models.CharField(max_length=200)
    published_date = models.DateField()
    author = models.ForeignKey(Author, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=6, decimal_places=2)


Key Points:

Each field represents a column in the table.

Field types (e.g., CharField, EmailField, DecimalField) determine the type of data stored.

Relationships:
ForeignKey (many-to-one): Links Book to Author.

on_delete=models.CASCADE: Deletes books if the author is deleted.

## Migrations:


Migrations are used to apply changes to the database schema.

Steps:
CreateMigrations:    **python manage.py makemigrations**

-- This generates migration files based on your model changes.

Apply Migrations:    **python manage.py migrate**

-- This applies the migration files to the database and creates/updates tables.

Check Migration Status:  **python manage.py showmigrations**




In [ ]:
python manage.py makemigrations

python manage.py migrate

python manage.py showmigrations

## Querying the Database
Django ORM uses querysets to retrieve data from the database.

### Examples:

In [ ]:
# Insert a new author
author = Author(name="Jane Austen", email="jane.austen@example.com")
author.save()

# Insert a new book
book = Book(title="Pride and Prejudice", published_date="1813-01-28", author=author, price=29.99)
book.save()


### Fetching Data:

In [ ]:
# Get all authors
authors = Author.objects.all()

# Filter books by price
expensive_books = Book.objects.filter(price__gt=20.00)

# Get a single object by ID
author = Author.objects.get(id=1)

# Fetch related objects
books_by_author = author.book_set.all()  # Access related books using the reverse relationship


### Updating Data:

In [ ]:
# Update an author's email
author = Author.objects.get(id=1)
author.email = "new.email@example.com"
author.save()


### Deleting Data:


In [ ]:
# Delete a book
book = Book.objects.get(id=1)
book.delete()


### Advanced Querying:

a) Filtering:

In [ ]:
# Books published in 2020
books_2020 = Book.objects.filter(published_date__year=2020)

# Authors whose name contains "Jane"
authors_named_jane = Author.objects.filter(name__icontains="Jane")


b) Aggregation:

In [ ]:
from django.db.models import Avg, Count, Max, Min

# Average price of all books
average_price = Book.objects.aggregate(Avg('price'))

# Total number of books
total_books = Book.objects.aggregate(Count('id'))


c) Ordering:

In [ ]:
# Order books by price (ascending)
books_by_price = Book.objects.order_by('price')

# Order by price (descending)
books_by_price_desc = Book.objects.order_by('-price')


## Relationships and Joins:

a) One-to-Many (ForeignKey):

In [ ]:
# Fetch all books by a specific author
jane_books = Book.objects.filter(author__name="Jane Austen")


b) Many-to-Many:


In [ ]:
# models.py

class Category(models.Model):
    name = models.CharField(max_length=100)

class Book(models.Model):
    title = models.CharField(max_length=200)
    categories = models.ManyToManyField(Category)

# Assigning categories to a book
fiction = Category.objects.create(name="Fiction")
romance = Category.objects.create(name="Romance")

book = Book.objects.create(title="Emma")
book.categories.add(fiction, romance)


c) Prefetch and Select Related:

To optimize queries and reduce database hits:

In [ ]:
# Fetch books with authors (joins)
books = Book.objects.select_related('author')

# Fetch books with categories (many-to-many relationships)
books = Book.objects.prefetch_related('categories')


### Raw SQL (If Needed)
In some cases, you might want to execute raw SQL queries:

In [ ]:
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM book WHERE price > %s", [20])
    rows = cursor.fetchall()


### Custom QuerySets
Define custom methods for querying:

In [ ]:
# models.py

class BookQuerySet(models.QuerySet):
    def expensive(self):
        return self.filter(price__gt=50)

class Book(models.Model):
    title = models.CharField(max_length=200)
    price = models.DecimalField(max_digits=6, decimal_places=2)
    objects = BookQuerySet.as_manager()

# Usage
expensive_books = Book.objects.expensive()


### Transactions
Django supports atomic transactions:

In [ ]:
from django.db import transaction

try:
    with transaction.atomic():
        author = Author.objects.create(name="Charles Dickens", email="charles.d@example.com")
        Book.objects.create(title="Great Expectations", author=author, price=19.99)
except Exception as e:
    print("Transaction failed:", e)


Summary:
Define Models: Use Django's ORM to represent database tables.

Migrate Changes: Sync models with the database using migrations.

Query Data: Use QuerySet for CRUD operations.

Optimize Queries: Use select_related, prefetch_related, and aggregation functions.

Leverage Relationships: Use Django ORM to handle relationships naturally.

By mastering these concepts, you'll effectively manage databases in Django applications. Practice with small projects to reinforce your understanding.    